In [1]:
from __future__ import print_function

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import mne
import copy

import ecogtools

In [2]:
%matplotlib inline

In [3]:
patient_num = "2003"

channels = ['LTG1', 'LTG2','LTG3','LTG4','LTG5',
           'LTG6','LTG7','LTG8','LTG9','LTG10',
           'LTG11','LTG12','LTG13','LTG14','LTG15',
           'LTG16','LTG17','LTG18','LTG19','LTG20',
           'LTG21','LTG22','LTG23','LTG24','LTG25',
           'LTG26','LTG27','LTG28','LTG29','LTG30',
           'LTG31','LTG32']

In [4]:
data = ecogtools.bio_motion_task_single_trial(patient_num)

Extracting edf Parameters from patient_2003/john_2003.edf...
Setting channel info structure...
Creating Raw.info structure...
Ready.


In [5]:
# Get the times for task from behavioral data
tmin = int(data.trig_and_behav.loc[0, 'trigger_time']-10)
tmax = int(data.trig_and_behav.loc[107, 'trigger_time']+10)

# Get only physiology data that we actually want to look at for task.
data.phys.crop(tmin=tmin, tmax=tmax)

<RawEDF  |  john_2003.edf, n_channels x n_times : 129 x 984001 (492.0 sec), ~221 kB, data not loaded>

In [6]:
# Load data and pick only channels that we care about.
data.phys.load_data()
data.phys.pick_channels(channels)

Reading 0 ... 984000  =      0.000 ...   492.000 secs...


<RawEDF  |  john_2003.edf, n_channels x n_times : 32 x 984001 (492.0 sec), ~240.3 MB, data loaded>

In [7]:
# Create epochs object for trial viz and averaging.
data.initialize_epochs_object(channels, tmin=-6., tmax=6., baseline=None)

132 matching events found
0 projection items activated
Loading data for 132 events and 24001 original time points ...
30 bad epochs dropped


In [35]:
# Create evoked for Question Start
bio_dir_evoked = data.create_evoked("Direction-Bio/trial_type")
contr_dir_evoked = data.create_evoked("Direction-Control/trial_type")
QS_evoked = mne.combine_evoked([contr_dir_evoked, bio_dir_evoked], weights=[1, -1])


In [36]:
# BANDPASS FILTER: Define frequencies, cycles and bandwidth
freqs = np.linspace(50, 150, 25)
n_cycles = freqs/2.0
time_bandwidth = 8.0

In [37]:
# Calculate power for Q Start high gamma ()
contr_dir_power = mne.time_frequency.tfr_multitaper(data.epochs['Direction-Control/trial_type'], freqs=freqs, n_cycles=n_cycles,
                       time_bandwidth=time_bandwidth, return_itc=False)
bio_dir_power = mne.time_frequency.tfr_multitaper(data.epochs['Direction-Bio/trial_type'], freqs=freqs, n_cycles=n_cycles,
                       time_bandwidth=time_bandwidth, return_itc=False)
QS_power = data.compute_diff_power(contr_dir_power, bio_dir_power)

In [38]:
# Copy data class for Filter-Hilbert Transform
data_filt = copy.copy(data)
data_filt.phys.filter(80, 160)
data_filt.phys.apply_hilbert(envelope=True)

Setting up band-pass filter from 80 - 1.6e+02 Hz
l_trans_bandwidth chosen to be 20.0 Hz
h_trans_bandwidth chosen to be 40.0 Hz
Filter length of 660 samples (0.330 sec) selected


<RawEDF  |  john_2003.edf, n_channels x n_times : 32 x 984001 (492.0 sec), ~240.3 MB, data loaded>

In [39]:
# New epochs object for filtered/HT data
data_filt.initialize_epochs_object(channels, tmin=-6., tmax=6., baseline=None)

132 matching events found
0 projection items activated
Loading data for 132 events and 24001 original time points ...
30 bad epochs dropped


In [40]:
# Evoked Power for Question Start sad-neutral/happy
bio_dir_evoked_filt = data_filt.create_evoked("Direction-Bio/trial_type")
contr_dir_evoked_filt = data_filt.create_evoked("Direction-Control/trial_type")
QS_filt = mne.combine_evoked([contr_dir_evoked_filt, bio_dir_evoked_filt], weights = [1, -1])

In [8]:
# Define frequencies, cycles and bandwidth
freqs = np.geomspace(1, 50, 50)
n_cycles = freqs/2.0
print(freqs)

[  1.           1.08311073   1.17312885   1.27062844   1.37623129
   1.49061088   1.61449663   1.74867862   1.89401257   2.05142534
   2.22192079   2.40658624   2.60659937   2.82323575   3.05787692
   3.3120193    3.58728363   3.88542538   4.20834591   4.5581046
   4.93693199   5.347244     5.79165734   6.27300619   6.7943603
   7.35904453   7.97066007   8.63310743   9.35061127  10.12774737
  10.96947182  11.88115261  12.86860384  13.93812287  15.0965304
  16.35121402  17.71017531  19.18208087  20.77631756  22.50305243
  24.37329749  26.39897997  28.5930184   30.96940496  33.54329473
  36.33110236  39.3506067   42.62106425  46.1633319   50.        ]


In [42]:
# Calculate power for Question Start low freq
low_contr_dir_power = mne.time_frequency.tfr_morlet(data.epochs['Direction-Control/trial_type'], freqs=freqs, n_cycles=n_cycles,
                    return_itc=False)
low_bio_dir_power = mne.time_frequency.tfr_morlet(data.epochs['Direction-Bio/trial_type'], freqs=freqs, n_cycles=n_cycles,
                    return_itc=False)
low_QS_power = data.compute_diff_power(low_contr_dir_power, low_bio_dir_power)

In [43]:
# Copy data class for Filter-Hilbert Transform
data_filt = copy.copy(data)
data_filt.phys.filter(1, 80)
data_filt.phys.apply_hilbert(envelope=True)

Setting up band-pass filter from 1 - 80 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 20.0 Hz
Filter length of 13200 samples (6.600 sec) selected


<RawEDF  |  john_2003.edf, n_channels x n_times : 32 x 984001 (492.0 sec), ~240.3 MB, data loaded>

In [44]:
# New epochs object for filtered/HT data
data_filt.initialize_epochs_object(channels, tmin=-6., tmax=6., baseline=None)

132 matching events found
0 projection items activated
Loading data for 132 events and 24001 original time points ...
30 bad epochs dropped


Plots


In [47]:
# Choose channels and define parameters for all functions
channel_of_interest = "LTG31"
index = channels.index(channel_of_interest)
COI = [index]

# Times and baseline periods for QS/TR
# Times and baseline periods for QS/TR
QS_times = {"tmin":-1., "tmax":4.}
TR_times = {"tmin":-4., "tmax":1.}
QS_baseline = (-1., 0)
TR_baseline = (0., .5)

ERROR! Session/line number was not unique in database. History logging moved to new session 123


In [48]:
%matplotlib inline

In [ ]:
# Times and baseline periods for QS

for channel in channels:
    channel_of_interest = channel
    index = channels.index(channel_of_interest)
    COI = [index]
    
    f, axes = plt.subplots(4, 3, figsize=(15,12))

    bio_dir_evoked.crop(**QS_times).plot(COI, axes=axes[0,0], show=False, titles="Bio Motion");
    contr_dir_evoked.crop(**QS_times).plot(COI, axes=axes[0, 1], show=False, titles="Control Motion");
    QS_evoked.crop(**QS_times).plot(COI, axes=axes[0, 2], show=False, titles="COMBINED");

    bio_dir_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[1, 0], show=False, colorbar=False);
    contr_dir_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[1, 1], show=False, colorbar=False);
    QS_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[1, 2], show=False, colorbar=False);

    low_bio_dir_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[2, 0], show=False, colorbar=False);
    low_contr_dir_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[2, 1], show=False, colorbar=False);
    low_QS_power.plot(COI, baseline=QS_baseline, mode='ratio', dB=True, vmin=-5., vmax=5., **QS_times, axes=axes[2, 2], show=False, colorbar=False);    
        
    bio_dir_evoked_filt.crop(**QS_times).plot(COI, axes=axes[3, 0], show=False);
    contr_dir_evoked_filt.crop(**QS_times).plot(COI, axes=axes[3, 1], show=False);
    QS_filt.crop(**QS_times).plot(COI, axes=axes[3, 2], show=False);

    f.savefig("patient_"+patient_num+'/Bio_Motion_plots/' + channel_of_interest + ".png")



In [24]:
data.trig_and_behav.to_csv('2003_data.csv')

In [25]:
data.epochs

<Epochs  |  n_events : 102 (all good), tmin : -6.0 (s), tmax : 6.0 (s), baseline : None, ~597.7 MB, data loaded,
 'Direction-Bio/time_of_resp': 15, 'Direction-Bio/trial_type': 18, 'Direction-Control/time_of_resp': 16, 'Direction-Control/trial_type': 18, 'Instrumental-Bio/time_of_resp': 9, 'Instrumental-Bio/trial_type': 10, 'Instrumental-Control/time_of_resp': 7, 'Instrumental-Control/trial_type': 9>

In [26]:
data.trig_and_behav

,action,correct_resp,is_type,resp_time,response,trial_type,trigger_name,cpu_trigger_time,task,trigger,trigger_time,trigger_index
0,Paint_45L,False,True,NaN,timeout,Direction,mov_start,56.548273,Bio_Motion,2,960.9076,1921815
1,Paint_45L,False,True,NaN,timeout,Direction,time_of_resp,57.000000,Bio_Motion,17,970.9076,1941815
2,Saw_45R,False,False,8.981997,left,Direction,mov_start,68.861094,Bio_Motion,2,973.2204,1946440
3,Saw_45R,False,False,8.981997,left,Direction,time_of_resp,77.843091,Bio_Motion,5,982.2024,1964404
4,Row_45L,False,True,7.482348,right,Direction,mov_start,79.935128,Bio_Motion,2,984.2945,1968588
5,Row_45L,False,True,7.482348,right,Direction,time_of_resp,87.417476,Bio_Motion,5,991.7768,1983553
6,Saw_0MM,True,False,3.588461,down,Direction,mov_start,89.506558,Bio_Motion,2,993.8659,1987731
7,Saw_0MM,True,False,3.588461,down,Direction,time_of_resp,93.095019,Bio_Motion,5,997.4544,1994908
8,Drive_45L,False,True,8.168387,down,Direction,mov_start,95.183875,Bio_Motion,2,999.5432,1999086
9,Drive_45L,False,True,8.168387,down,Direction,time_of_resp,103.352262,Bio_Motion,5,1007.7116,2015423


In [27]:
bio_dir_evoked_filt

<Evoked  |  comment : 'Direction-Bio/trial_type', kind : average, time : [-1.000000, 4.000000], n_epochs : 18, n_channels x n_times : 32 x 10001, ~2.5 MB>